In [1]:
from tensorflow.keras.models import load_model
from tqdm.notebook import tqdm
from numpy import load
import numpy as np
import pandas as pd
import json

In [167]:
round(float(1), 3)

1.0

In [168]:
def calculate_accuracy(true_words, predicted_words):
    total_words = len(true_words)
    correct_predictions = sum(1 for true, pred in zip(true_words, predicted_words) if true == pred)
    accuracy = correct_predictions / total_words
    return round(float(accuracy), 3)

def calculate_precision(true_words, predicted_words):
    true_positives = sum(1 for true, pred in zip(true_words, predicted_words) if true == pred)
    predicted_positives = sum(1 for pred in predicted_words if pred in true_words)

    if predicted_positives == 0:
        precision = 0
    else:
        precision = true_positives / predicted_positives

    return round(precision, 3)


def calculate_recall(true_words, predicted_words):
    true_positives = sum(1 for true, pred in zip(true_words, predicted_words) if true == pred)
    actual_positives = len(true_words)

    if actual_positives == 0:
        recall = 0
    else:
        recall = true_positives / actual_positives

    return round(recall, 3)

# Predict the label from model and find which letter it is
def find_preds_letters(model, test_X, tuples_dict):
        preds_letters = []

        preds = model.predict(test_X)

        for pred in preds:
                max_value = pred.max()
                idx = np.where(pred==max_value)[0][0]
                preds_letters.append(tuples_dict[str(idx)])
        return preds_letters

# Find accuraccy with Normalized Hamming Distance
def calculate_normalized_hamming_distance(preds_letters, labels, test_filenames):
    final_preds = {}
    words = []
    pred_words = []

    counter_correct = 0
    counter_word = 0
    word = []
    pred_word = []

    filename_old = test_filenames[0]
    for p, l, f in zip(preds_letters, labels, test_filenames):
        
        if f == filename_old:
            if p == l:
                counter_correct = counter_correct +1
                counter_word = counter_word +1 
                word.append(l)
                pred_word.append(p)
            else:
                counter_word = counter_word +1 
                word.append(l)
                pred_word.append(p)
        else:
            temp_dict = {
                'accuracy' : counter_correct/counter_word,
                'word' : ''.join(word),
                'pred_word' : ''.join(pred_word)
            }
            final_preds[filename_old] = temp_dict
            words.append(''.join(word))
            pred_words.append(''.join(pred_word))

            filename_old = f
            counter_correct = 0
            counter_word = 0
            word = []
            pred_word = []

            if p == l:
                counter_correct = counter_correct +1
                counter_word = counter_word +1 
                word.append(l)
                pred_word.append(p)
            else:
                counter_word = counter_word +1 
                word.append(l)
                pred_word.append(p)
    return final_preds, words, pred_words

In [155]:
# Define paths
csv_path = "data/csv/written_name_test_v2.csv"
gray_data_path = "data/Gray/"
json_data_path = "data/Json/"
models_path = 'models/'
models_names = [
        "model_batch_size_86_epochs_20_train_30000",
        "model_batch_size_86_epochs_30_old_architecture",
        "model_batch_size_86_epochs_30_train_60000",
        "model_batch_size_86_epochs_30_train_60000_imagegenerator",
        "model_batch_size_86_epochs_30_train_90000"

]

#Import numpy files
test_X = load(gray_data_path+'test_X.npy')
test_Y = load(gray_data_path+'test_Y.npy')
test_filenames = load(gray_data_path+'test_filenames.npy')

test_X =test_X[0:500]
test_Y =test_Y[0:500]
test_filenames = test_filenames[0:500]



# Import Json
with open(json_data_path+'tuples.json', 'r') as openfile:
 
    # Reading from json file
    tuples_dict = json.load(openfile)

In [178]:
accuraccy_scores = []
precision_scores = []
recall_scores = []
model_names = []

for model_name in models_names:
    # Import model
    model = load_model(models_path+model_name+".h5")

    preds_letters = find_preds_letters(model, test_X, tuples_dict)
    final_preds, words, pred_words = calculate_normalized_hamming_distance(preds_letters, labels, test_filenames)

    accuraccy_scores.append(calculate_accuracy(words, pred_words))
    precision_scores.append(calculate_precision(words, pred_words))
    recall_scores.append(calculate_recall(words, pred_words))
    model_names.append(model_name)

dict = {
    'Model' : model_names,
    'Accuraccy_score': accuraccy_scores,
    'Precision_score':precision_scores,
    'Recall_score':recall_scores
}
df = pd.DataFrame.from_dict(dict)


Accuraccy score: 0.26, Precision Score: 1.0, Recall Score: 0.26
Accuraccy score: 0.26, Precision Score: 1.0, Recall Score: 0.26
Accuraccy score: 0.26, Precision Score: 1.0, Recall Score: 0.26
Accuraccy score: 0.26, Precision Score: 1.0, Recall Score: 0.26
Accuraccy score: 0.26, Precision Score: 1.0, Recall Score: 0.26


In [179]:

df

,Model,Accuraccy_score,Precision_score,Recall_score
0,model_batch_size_86_epochs_20_train_30000,0.000,0.0,0.000
1,model_batch_size_86_epochs_30_old_architecture,0.429,1.0,0.429
2,model_batch_size_86_epochs_30_train_60000,0.506,1.0,0.506
3,model_batch_size_86_epochs_30_train_60000_imag...,0.026,1.0,0.026
4,model_batch_size_86_epochs_30_train_90000,0.260,1.0,0.260


In [124]:
print(accuraccy_score)
print(precision_score)
print(recall_score)

0.2597402597402597
1.0
0.2597402597402597


In [142]:
df = pd.DataFrame()

,accuraccy_score,precision_score,recall_score
0,0.25974,1.0,0.25974
